# Szablony klas - templates
Konieczność implementowania funkcjonalności dla każdego typu oddzielnie może być kłopotliwa. Język C++ (niektóre inne języki też) wprowadza mechanizm szablonów - templejtów (generyki?).

Mechanizm szablonów umożliwia tworzenie rozwiązań niezależnych od typu i pozwalając na nieduplikowanie kodu. Rozwinięcie szablonów odbywa się w momencie kompilacji, tak jak znane z C makra. Konsekwencją jest pojecie meta-programowania. Można przyjąć, że meta-programowanie to programowanie instrukcji dla kompilatora w celu napisania dla nas kodu, czyli takie programowanie programowania. Ciekawą konsekwencją jest uznanie mechanizmu szablonów obecnego w C++ za oddzielny język programowania!

Szablony mogą być oparte o typ (klasę) lub wartość całkowita. Mogą być stosowane w stosunku do funkcji, klas i metod, w rożnych konfiguracjach. Sam mechanizm szablonów może być niezmiernie przydatny, ale jak ze wszystkim, łatwo przesadzić upychając szablony na siłę i doprowadzić do małej czytelności kodu. Dodatkowo błędy kompilacji wynikające z zastosowania szablonów potrafią być przytłaczające i są natchnieniem branżowych memów, jak na przykład ten:

![template meme](https://i.redd.it/nsb2m3472lx31.png)

**Uwaga:** Do definicji szablonu opartego o typ (klasę) można użyć zarówno słowa kluczowego *class* jak i *typename*. Nie ma w zasadzie różnicy, czego się użyje (wynika to z historii C++).

## Po co:
Zaczniemy od prostego przykładu, którego celem będzie przekonanie nas, że szablony mogą być dla nas przydatne. Powiedzmy, że chcielibyśmy napisać funkcję, która była by w stanie obsłużyć wszystkie możliwe typy, dla których zdefiniowano operację mnożenia. Chcemy móc podnieść wszystko do kwadratu:

In [2]:
#include <iostream>
using namespace std;

In [3]:
int square(int x){  return x*x;}

In [4]:
short int square(short int x){  return x*x;}
unsigned short int square(unsigned short int x){  return x*x;}
long square(long x){  return x*x;}
unsigned long square(unsigned long x){  return x*x;}
long long square(long long x){  return x*x;}
unsigned long long square(unsigned long long x){  return x*x;}
float square(float x){  return x*x;}
double square(double x){  return x*x;}
// i tak dalej, i jeszcze dla naszych typow ...

input_line_6:2:31: error: function definition is not allowed here
 short int square(short int x){  return x*x;}
                              ^
input_line_6:3:48: error: function definition is not allowed here
unsigned short int square(unsigned short int x){  return x*x;}
                                               ^
input_line_6:4:20: error: function definition is not allowed here
long square(long x){  return x*x;}
                   ^
input_line_6:5:38: error: function definition is not allowed here
unsigned long square(unsigned long x){  return x*x;}
                                     ^
input_line_6:6:30: error: function definition is not allowed here
long long square(long long x){  return x*x;}
                             ^
input_line_6:7:48: error: function definition is not allowed here
unsigned long long square(unsigned long long x){  return x*x;}
                                               ^
input_line_6:8:22: error: function definition is not allowed here
float square

ename: evalue

In [5]:
int a = 5;
//...
cout << square(a) << endl; 

25


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f2f0ad4a540


By podnieść dramatyzm dodajmy do typów wbudowanych, te, które sami napisaliśmy.

In [6]:
#include <iostream>
using namespace std;

class complex{//liczba zespolona na int
public:
  complex(int r, int i) : real(r), imag(i) {}
  const int& c_real(){return real;}
  const int& c_imag(){return imag;}
private:
  int real;
  int imag;
};

In [7]:
complex square(complex& a){
  return complex(a.c_real()*a.c_real(),
                a.c_imag()*a.c_imag());
}

In [8]:
complex aa(2,3);
complex b = square(aa);
//...
cout << aa.c_real() << " " << aa.c_imag() << " "
     << b.c_real() << " " << b.c_imag() << endl;

2 3 4 9


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f2f0ad4a540


Widzimy, więc, że jedna mała funkcja wymaga dużego nakładu pracy. Dlaczego by nie zaprząc do tej pracy kompilatora. Tzn. stworzyć mechanizm, który wygeneruje potrzebne metody w trakcie kompilacji na podstawie dostarczonej przez nas recepty - szablonu.

## Szablony
Ogólna składnia dla funkcji:

In [ ]:
template <class T>
typ_funkcji fun (argumenty)
{
    Cialo funkcji
}

I dla klasy:

In [ ]:
template <class T>
class List
{
    Cialo klasy
};

## Szablony funkcji
### Szablon oparty o typ
Zacznijmy of szablonu funkcji opartej o typ (klasę), np funkcję, która oblicza i zwraca kwadrat zmiennej dowolnego typu:

In [1]:
#include <iostream>
using namespace std;

Podnieść do kwadratu wartość

In [2]:
template <class T>
T sq(const T& x)
{
    return x*x; // assumption is T knows what * is
}

In [3]:
double a = 5;
int    b = 9;
char   c = 33;

Wywolanie szablonu odbywa się jak wywołanie każdej funkcji, z ta roznicą, że nalezy podać typ wywoływanej funkcji, o tak:

In [4]:
wartość_zwracana = nazwa_funkcji<typ>(argumenty)

input_line_6:2:2: error: use of undeclared identifier 'wartość_zwracana'
 wartość_zwracana = nazwa_funkcji<typ>(argumenty)
 ^
input_line_6:2:23: error: use of undeclared identifier 'nazwa_funkcji'
 wartość_zwracana = nazwa_funkcji<typ>(argumenty)
                    ^
input_line_6:2:37: error: use of undeclared identifier 'typ'
 wartość_zwracana = nazwa_funkcji<typ>(argumenty)
                                  ^
input_line_6:2:42: error: use of undeclared identifier 'argumenty'
 wartość_zwracana = nazwa_funkcji<typ>(argumenty)
                                       ^


ename: evalue

In [5]:
cout << sq<double>(a) << " " << sq<int>(b) << " " << sq<char>(c) << endl;

25 81 A


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fde4b7f8540


In [6]:
a = 2.55154125;
cout << sq<int>(a) << endl;

4


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fde4b7f8540


W tym przypadku kompilator jest "domyślny" nie musimy podawać typu. jest tak w przypadku funkcji, których typ mozna wydedukować z listy argumentów. Nie zawsze musi być to mozliwe:

In [7]:
a = 5;
cout << sq(a) << " " << sq(b) << " " << sq(c) << endl;

25 81 A


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fde4b7f8540


Funkcja *sq()* wymaga aby prtraktowany przez nią typ implementował operację \*. W przypadku naszej klasy należy więc dopisać taką operację:

In [3]:
class complex{//liczba zespolona na int
public:
  complex(const int& r, const int& i) : real(r), imag(i) {}
  const int& c_real() const {return real;}
  const int& c_imag() const {return imag;}
private:
  int real;
  int imag;
public:
  complex operator *(const complex& b) const // complex ni wie co to *
  {
    return complex(this->real * b.c_real(),
                   this->imag * b.c_imag());
  }
};

In [4]:
complex ca(4,5);
complex cb = sq<complex>(ca);

cout << ca.c_real() << " " << ca.c_imag() << " "
       << cb.c_real() << " " << cb.c_imag() << endl;

4 5 16 25


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f7766700540


In [5]:
auto cc = sq(ca);

In [6]:
cout << cc.c_real() << " " << cc.c_imag() << endl;

16 25


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f7766700540


Przypomnijmy sobie teraz, że funkcje mają adresy. Tzn. są gdzieś w pamięci. Czy *sq\\<int\>()* to to samo co *sq\\<double\>()* ?

In [1]:
#include <stdio.h>

In [2]:
template<typename T>
T square(T& x){return x*x;}

In [3]:
printf("%p %p \n", square<int>, square<double>);   

0x7feb8445f0e0 0x7feb8445f0c0 


(int) 31


Powinno być oczywistym, że nie.

### Szablon oparty o wartość calkowitą
Szablon może być oprty o wartość całkowitą. Przykład poniżej zwraca argument pomnożony przez argument szablonu (troche to bez sensu, ale niech będzie):

In [1]:
#include <iostream>
using namespace std;

In [2]:
template <int V>
int fun(int x){
    return V * x;
}

In [3]:
cout << fun<90>(2) << endl;

180


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f390957a540


In [4]:
const int b = 5;
cout << fun<b>(2) << endl;

10


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f390957a540


Może być to przydatne do ustalenie rozmiaru statycznej tablicy:

In [5]:
template <int N>
int fun2(int x)
{
    int tab[N];
    
    // tu algorytm oparty o tablice
    for(int i=0; i<N; ++i)
    {
        tab[i] = i * N;
    }
    
    return x * tab[N-1];
}

In [7]:
int n = 9;
cout << fun2<7>(n) << endl;

input_line_10:2:6: error: redefinition of 'n' with a different type: 'int' vs 'const int'
 int n = 9;
     ^
input_line_9:2:12: note: previous definition is here
 const int n = 9;
           ^


ename: evalue

### Wiele argumentów szablonu
Argumentów szablonu może być więcej niż jeden. Zwróc uwage, że te funkcje są rózne:

In [1]:
#include <iostream>
using namespace std;

In [2]:
template <class T, int V>
T fun(T x){
    return V * x;
}

template <int V, class T>
T fun(T x){
    return V * x;
}

In [3]:
cout << fun<double, 2>(7) << " " << fun<5, double>(5) << endl;

14 25


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe917c9a540


In [4]:
template <typename T, int N>
T fun2(const T& x)
{
    T tab[N];
    
    // tu algorytm oparty o tablice
    //tab[N-1] = N * N; // ?
    
    return x;
}

In [5]:
cout << fun2<double, 10>(10) << endl;

10


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe917c9a540


Kolejność podawania argumentów szablonu ma znaczenie. Pokażemy to na przykładzie prostej funkcji podnoszącej typ T do potęgi całkowitej:

In [1]:
#include <iostream>
using namespace std;

In [2]:
template<typename T, int N>
T pow(T x)
{
  cout << "funkcja type, int" << endl;
  T res=1;
  for(int i=0; i<N; ++i)
    res *= x;
  return res;
}

Oraz:

In [3]:
template<int N, typename T>
T pow(T x)
{
  cout << "funkcja int, type" << endl;
  T res=1;
  for(int i=0; i<N; ++i)
    res *= x;
  return res;
}

In [4]:
double a=9;
cout << pow<double, 2>(a) << endl;
cout << pow<2>(a) << endl;

funkcja type, int
81
funkcja int, type
81


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f2cf25a2540


### Specjalizacja
W przypadku konkretnych typów może zajść konieczność specjalizacji, tj. dokładnego okreslenia co ma się zdarzyć:

In [ ]:
#include <iostream>
using namespace std;

In [ ]:
template<typename T>
T fun(T x)
{
  return 2*x;
}

template<>
string fun<string>(string x)
{
  return x + " " + x;
}

In [ ]:
cout << fun(5.0) << " " << endl;
string s = "Ala ma kota!!";
cout << fun(s) << endl;

## Szablon klasy
Podobnie jak funkcje można "ztemplejtować" metody klasy, jak i same klasy. Powiedzmy, że chcielibyśmy zaprojektować typ complex dla wartośći całkowitych i zmiennoprzecinkowych:

In [ ]:
class complex_int{//liczba zespolona na int
public:
  complex_int(int r, int i) : real(r), imag(i) {}
  const int& c_real(){return real;}
  const int& c_imag(){return imag;}
private:
  int real;
  int imag;
};

class complex_double{//liczba zespolona na int
public:
  complex_double(double r, double i) : real(r), imag(i) {}
  const double& c_real(){return real;}
  const double& c_imag(){return imag;}
private:
  double real;
  double imag;
};

Czemu by nie wykorzystać mechanizmu szablonów:

In [ ]:
#include <iostream>
#include <vector>
using namespace std;

In [ ]:
template<class T>
class complex{
public:
  complex(const T& r, const T& i) : real(r), imag(i) {}
  const T& c_real(){return real;}
  const T& c_imag(){return imag;}
    
  void fun();
private:
  T real;
  T imag;
};

In [ ]:
template <typename T>
void complex<T>::fun()
{
    cout << "Zawolano funa" << endl;
}

Widzimy już, jakie było znaczenie \<\>, które wudzieliśmy w vector<>:

In [ ]:
complex<int> a(9,8);
complex<double> b(3.141592, 2.73);

In [ ]:
cout << a.c_real() << " " << a.c_imag() << endl;
cout << b.c_real() << " " << b.c_imag() << endl;

In [ ]:
vector<double> dd;

In [ ]:
vector<vector<complex<int> > > aaa;// bo >> jest operatorem

W przeciwieństwie do dziedziczenie, obiekty wynikające z rozinięcia szablonu różnymi argumentami nie mają ze sobą nic wspólnego:

In [ ]:
class_name < int > != class_name < double >

In [ ]:
#include <iostream>
using namespace std;

In [ ]:
template<class T>
class complex{
public:
  complex(const T& r, const T& i) : real(r), imag(i) {}
  const T& c_real(){return real;}
  const T& c_imag(){return imag;}
  void fun();
private:
  T real;
  T imag;
};

In [ ]:
template<class T>
void complex<T>::fun()
{
  cout << sizeof(T) << endl;
  cout << sizeof(*this) << endl;
}

In [ ]:
complex<int> a(9,8), b(5,6);
complex<double> c(3.141592, 2.73);
//c = b;
a.fun();
b.fun();
c.fun();

## Metaprogramowanie - język w języku

* Okazuje się, że mechanizm szablonów w C++ jest osobnym językiem
* Tak, wewnątrz C++ zaszyto inny język
* Turing Kompletny ...
* Odkryto to przypadkiem
* I tak, ma zastosowania
* Pozwala na wykonanie ”programu”w czasie kompilacji - metaprogramowanie!

### Silnia:

In [ ]:
#include <iostream>
using namespace std;

In [ ]:
unsigned int factorial ( unsigned int n )
{
    cout << " Called with n = " << n << endl ;
    return n == 0 ? 1 : n * factorial ( n - 1) ;
}

In [ ]:
cout <<" Finally the factorial is : " << factorial (4) << endl ;

### Silnia trochę inaczej:

In [ ]:
#include <iostream>
using namespace std;

In [ ]:
template < unsigned int n >
class factorial {
    public :
    static const unsigned long long value = n * factorial <n -1 >:: value ;
};

In [ ]:
template <>
class factorial <0 > {
    public :
    static const unsigned long long value = 1;
};

In [ ]:
std :: cout << factorial <5 >:: value << '\n' ;

## Częścowa specjalizacja

In [1]:
...

input_line_3:2:2: error: expected expression
 ...
 ^


ename: evalue